In [28]:
from haystack.components.agents import Agent
from haystack.components.generators.chat import HuggingFaceLocalChatGenerator
from haystack.components.websearch import SerperDevWebSearch
from haystack.dataclasses import ChatMessage
from haystack.tools.component_tool import ComponentTool
from duckduckgo_api_haystack import DuckduckgoApiWebSearch

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from chatcore.tools.ifc_tool import ifc_entity_tool, ifc_query_tool, ifc_finetune_tool, no_call_tool, IfcToolCallAssistant
from chatcore.utils.config_loader import load_llm_config  


In [33]:
#llm_config = load_llm_config()
llm = HuggingFaceLocalChatGenerator(
        model="meta-llama/Llama-3.2-3B-Instruct",
        huggingface_pipeline_kwargs={
            "device_map": "auto",
            #"torch_dtype": llm_config["torch_dtype"],   
        },
        generation_kwargs={"max_new_tokens": 1024,
                           "temperature": 0.7,
                           "repetition_penalty": 1.1}
    )
llm.warm_up()   

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.
Device set to use cuda:0


In [24]:
# Create the agent with the web search tool
agent = Agent(chat_generator=llm, tools=[ifc_entity_tool, ifc_query_tool, ifc_finetune_tool, no_call_tool])
agent.warm_up()

ifc_file_path= "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Courses/CRBE/IFC/BIM4EEB-TUD-2x3.ifc"
# Run the agent with a query
result = agent.run(messages=[ChatMessage.from_user(f"What are the main ifcentities in the ifc file from {ifc_file_path}?")])

# Print the final response
print(result)

{'messages': [ChatMessage(_role=<ChatRole.USER: 'user'>, _content=[TextContent(text='What are the main ifcentities in the ifc file from C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Courses/CRBE/IFC/BIM4EEB-TUD-2x3.ifc?')], _name=None, _meta={}), ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='{"name": "ifc_entity_tool", "parameters": {"ifc_file_path": "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Courses/CRBE/IFC/BIM4EEB-TUD-2x3.ifc"}}')], _name=None, _meta={'finish_reason': 'stop', 'index': 0, 'model': 'meta-llama/Llama-3.2-3B-Instruct', 'usage': {'completion_tokens': 64, 'prompt_tokens': 504, 'total_tokens': 568}})], 'last_message': ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='{"name": "ifc_entity_tool", "parameters": {"ifc_file_path": "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Courses/CRBE/IFC/BIM4EEB-TUD-2x3.ifc"}}')], _name=None, _meta={'finish_reason': 'stop', 'index'

In [25]:
print(result["messages"][-1].text)

{"name": "ifc_entity_tool", "parameters": {"ifc_file_path": "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Courses/CRBE/IFC/BIM4EEB-TUD-2x3.ifc"}}


In [34]:
# Create the web search component
web_search = DuckduckgoApiWebSearch(top_k=3)

# Create the ComponentTool with simpler parameters
web_tool = ComponentTool(
    component=web_search,
    name="web_search",
    description="Search the web for current information like weather, news, or facts."
)

# Create the agent with the web tool
tool_calling_agent = Agent(
    chat_generator=llm,
    system_prompt="""You're a helpful agent. When asked about current information like weather, news, or facts, 
                     use the web_search tool to find the information and then summarize the findings.
                     When you get web search results, extract the relevant information and present it in a clear, 
                     concise manner.""",
    tools=[web_tool]
)
tool_calling_agent.warm_up()

# Run the agent with the user message
user_message = ChatMessage.from_user("How is the weather in Berlin?")
result = tool_calling_agent.run(messages=[user_message])

# Print the result - using .text instead of .content
print(result["messages"][-1].text)

{"name": "web_search", "parameters": {"query": "Berlin weather"}}


In [35]:
from pprint import pprint
pprint(result)

{'last_message': ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>,
                             _content=[TextContent(text='{"name": '
                                                        '"web_search", '
                                                        '"parameters": '
                                                        '{"query": "Berlin '
                                                        'weather"}}')],
                             _name=None,
                             _meta={'finish_reason': 'stop',
                                    'index': 0,
                                    'model': 'meta-llama/Llama-3.2-3B-Instruct',
                                    'usage': {'completion_tokens': 17,
                                              'prompt_tokens': 265,
                                              'total_tokens': 282}}),
 'messages': [ChatMessage(_role=<ChatRole.SYSTEM: 'system'>,
                          _content=[TextContent(text="You're a hel